In [2]:
# BEFORE DOING ANYTHING
# cd Desktop
# git pull

# .. then do stuff ..
# git add .
# git commit -m "your message"
# git push


import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#df = pd.read_csv("listings.csv")

# The top 10 most important features are:
# How many people the property accommodates
# The cleaning fee
# How many other listings the host has (and whether they are a multi-listing host)
# How many days are available to book out of the next 90
# The fee per extra person
# The number of reviews
# The number of bathrooms
# The security deposit
# If the property is in Westminster
# The minimum nights stay


raw_df = pd.read_csv("listings.csv.gz")


cols_to_drop = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'calendar_last_scraped']

df = raw_df.drop(cols_to_drop, axis=1)
df

/Users/ajay/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,experiences_offered,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,109,none,2008-06-27,NaN,NaN,NaN,f,1.0,1.0,t,...,f,f,strict_14_with_grace_period,t,f,1,1,0,0,0.02
1,344,none,2008-07-11,within a day,50%,NaN,f,1.0,1.0,t,...,t,f,flexible,f,f,1,1,0,0,0.15
2,2708,none,2008-09-16,within an hour,100%,NaN,t,2.0,2.0,t,...,t,f,strict_14_with_grace_period,f,f,2,0,2,0,0.33
3,2732,none,2008-09-17,within an hour,100%,NaN,t,2.0,2.0,t,...,f,f,strict_14_with_grace_period,f,f,2,1,1,0,0.19
4,2864,none,2008-09-25,NaN,NaN,NaN,f,1.0,1.0,t,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,NaN
5,5728,none,2009-03-05,within an hour,100%,NaN,t,8.0,8.0,t,...,t,f,moderate,f,f,4,1,3,0,2.32
6,5729,none,2009-03-05,within an hour,100%,NaN,t,8.0,8.0,t,...,t,f,moderate,f,f,4,1,3,0,1.60
7,5843,none,2009-03-05,within an hour,100%,NaN,t,8.0,8.0,t,...,t,f,moderate,f,f,4,1,3,0,1.03
8,6033,none,2009-03-28,within an hour,100%,NaN,f,14.0,14.0,t,...,t,f,strict_14_with_grace_period,f,f,5,2,3,0,0.18
9,6931,none,2008-09-16,within an hour,100%,NaN,t,2.0,2.0,t,...,t,f,strict_14_with_grace_period,f,f,2,0,2,0,0.14


In [ ]:
print "hello"